In [1]:
from qiskit import BasicAer, Aer, IBMQ
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua.components.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit.aqua.components.variational_forms import RY, RYRZ, SwapRZ
from qiskit.aqua.operators import WeightedPauliOperator, Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error

from qiskit.providers.aer import noise
provider = IBMQ.load_account()

import numpy as np
import matplotlib.pyplot as plt
from functools import partial

In [2]:
IBMQ.load_account()
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2020-08-07 14:33:09,489: Credentials are already in use. The existing account in the session will be replaced.
ibmqfactory.load_account:WARNING:2020-08-07 14:33:17,934: Credentials are already in use. The existing account in the session will be replaced.


In [3]:
#Classically solve for the lowest eigenvalue
def exact_solver(qubitOp):
    ee = ExactEigensolver(qubitOp)
    result = ee.run()
    ref = result['energy']
    print('Reference value: {}'.format(ref))
    return ref

In [4]:
def exact_solver(qubitOp):
    ee = ExactEigensolver(qubitOp)
    result = ee.run()
    ref = result['energy']
    print('Reference value: {}'.format(ref))
    return ref

# Define your function for computing the qubit operations of LiH
def compute_LiH_qubitOp(map_type, inter_dist, basis='sto3g'):
    
    # Specify details of our molecule
    driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 ' + str(inter_dist), unit=UnitsType.ANGSTROM, charge=0, spin=0, basis=basis)

    # Compute relevant 1 and 2 body integrals.
    molecule = driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    nuclear_repulsion_energy = molecule.nuclear_repulsion_energy
    
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
    print("# of electrons: {}".format(num_particles))
    print("# of spin orbitals: {}".format(num_spin_orbitals))

    # Please be aware that the idx here with respective to original idx
    freeze_list = [0]
    remove_list = [-3, -2] # negative number denotes the reverse order
    
    # Prepare full idx of freeze_list and remove_list
    # Convert all negative idx to positive
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    
    # Update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]

    # Prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
    # and if PARITY mapping is selected, reduction qubits
    energy_shift = 0.0
    qubit_reduction = True if map_type == 'parity' else False

    ferOp = FermionicOperator(h1=h1, h2=h2)
    if len(freeze_list) > 0:
        ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
        num_spin_orbitals -= len(freeze_list)
        num_particles -= len(freeze_list)
    if len(remove_list) > 0:
        ferOp = ferOp.fermion_mode_elimination(remove_list)
        num_spin_orbitals -= len(remove_list)

    qubitOp = ferOp.mapping(map_type=map_type,threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
    qubitOp.chop(10**-10)

    return qubitOp, num_spin_orbitals, num_particles, qubit_reduction,energy_shift,molecule.nuclear_repulsion_energy

qubitOp, num_spin_orbitals, num_particles, qubit_reduction,energy_shift,nuclear_repulsion_energy = compute_LiH_qubitOp('parity',1.6)
# Classically solve for the exact solution and use that as your reference value
ref = exact_solver(qubitOp)
print(ref)

HF energy: -8.854072040283647
# of electrons: 4
# of spin orbitals: 12


/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/algorithms/eigen_solvers/numpy_eigen_solver.py:242: DeprecationWarning: Deprecated class ExactEigensolver, use NumPyEigensolver.
  DeprecationWarning)


Reference value: -1.0770597457346958
-1.0770597457346958


/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/algorithms/eigen_solvers/eigen_solver_result.py:64: DeprecationWarning: energy deprecated, use eigenvalues property.
  warnings.warn('energy deprecated, use eigenvalues property.', DeprecationWarning)


In [5]:
dist = 1.6
qubitOp, num_spin_orbitals, num_particles, qubit_reduction,energy_shift,nuclear_repulsion_energy = compute_LiH_qubitOp('parity',dist)

HF energy: -8.854072040283645
# of electrons: 4
# of spin orbitals: 12


In [6]:
backend = Aer.get_backend("qasm_simulator")
chip_name = 'ibmq_essex'
device = provider.get_backend(chip_name)
coupling_map = device.configuration().coupling_map
noise_model =noise.device.basic_device_noise_model(device.properties())
basis_gates = noise_model.basis_gates

quantum_instance = QuantumInstance(backend=backend,
                                   shots=1000,
                                   basis_gates= basis_gates,
                                   coupling_map=coupling_map,
                                   noise_model=noise_model )

/home/codie/my_env/lib/python3.6/site-packages/qiskit/providers/aer/noise/device/basic_device_model.py:115: DeprecationWarning: This function is been deprecated and moved to a method of the`NoiseModel` class. For equivalent functionality use `NoiseModel.from_backend(properties, **kwargs).
  DeprecationWarning)


In [7]:
optimizer = COBYLA(maxiter=100)
num_qubits = 4
var_form = RY(num_qubits, depth=3,entangler_map=[[0,1],[1,2],[1,3]], entanglement='linear',entanglement_gate='cz')

vqe = VQE(qubitOp, var_form, optimizer=optimizer)

ret = vqe.run(quantum_instance)
vqe_result = np.real(ret['eigenvalue'])

print("VQE Result:", vqe_result)

/home/codie/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The qiskit.aqua.components.variational_forms.RY object is deprecated as of 0.7.0 and will be removed no sooner than 3 months after the release. You should use qiskit.circuit.library.RealAmplitudes (uses CX entangling) or qiskit.circuit.library.TwoLocal instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.r

/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

...

/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

VQE Result: -0.7851878566167899


/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_fn.py:209: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  ndigits=EVAL_SIG_DIGITS)
/home/codie/my_env/lib/python3.6/site-packages/qiskit/aqua/operators/state_fns/dict_state_f

In [8]:
print(vqe_result+nuclear_repulsion_energy+energy_shift)
print(ref+nuclear_repulsion_energy+energy_shift)

-7.589200154913018
-7.881072044030924
